In [1]:
dataname="synthetic"

patch_size=256 #size of the tiles to put into DB
data_size=[10000,100]
balance=.5
classes=[0,1] #what classes we expect to have in the data, in this case data without boxes and data with boxes

max_circles=10
max_squares=1
diameter_min=10
diameter_max=50

phases=["train","val"]

In [2]:
import random
import tables
import sys

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw


seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the cross validation setup
random.seed(seed) # set the seed
print(f"random seed (note down for reproducibility): {seed}")

random seed (note down for reproducibility): 4682126040759051305


In [3]:
img_dtype = tables.UInt8Atom()  # dtype in which the images will be saved, this indicates that images will be saved as unsigned int 8 bit, i.e., [0,255]

In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [5]:
%cd My\ Drive/PytorchDigitalPathology/visualization_densenet

/gdrive/My Drive/PytorchDigitalPathology/visualization_densenet


In [6]:
%matplotlib inline
storage={} #holder for future pytables

block_shape=np.array((patch_size,patch_size)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed


for phase,nimgs in zip(phases,data_size): #now for each of the phases, we'll loop through the files
    print(phase)
    
    totals=np.zeros(2) # we can to keep counts of all the classes in for in particular training, since we 

    hdf5_file = tables.open_file(f"./{dataname}_{phase}.pytable", mode='w') #open the respective pytable


    storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
                                              shape=np.append([0],block_shape), 
                                              chunkshape=np.append([1],block_shape),
                                              filters=filters)
    storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
                                              shape=[0], 
                                              chunkshape=[1],
                                              filters=filters)

    
    for filei in range(nimgs): #now for each of the files
        img=np.zeros((patch_size,patch_size))
        img = Image.fromarray(img)
        draw= ImageDraw.Draw(img)
        
        for i in range(np.random.randint(0,high=max_circles)):
            d=np.random.randint(diameter_min,diameter_max)
            ul=np.random.randint(diameter_min,patch_size-diameter_max,2)
            draw.ellipse(list(np.append(ul,ul+d)),fill=255)
    

        label=np.random.random()>balance
        if label:
            for i in range(np.random.randint(1,high=max_squares+1)):
                d=np.random.randint(diameter_min,diameter_max)
                ul=np.random.randint(diameter_min,patch_size-diameter_max,2)
                draw.rectangle(list(np.append(ul,ul+d)),fill=255)
                totals[1]+=1
        else:
            totals[0]+=1
            #add square
        
        del draw    

        storage["imgs"].append(np.array(img)[None,::])
        storage["labels"].append([np.uint8(label)]) #add the filename to the storage array
        
    #lastely, we should store the number of pixels
    npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
    npixels[:]=totals
    hdf5_file.close()
    
print("done")

train
val
done
